Calculate Shap values and draw

计算shap值以及绘图

In [1]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import myfunction as mf
path_data_raw = "C:/Users/dell/OneDrive/file/"
path_country_nc = "C:/Users/dell/OneDrive/file/nc"
path_one_spdb = 'C:/Users/dell/OneDrive/file/SPDB/'
drive_letter = 'E:'

path_pre = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/pretreatment/"
path_match = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/match/"
path_semdata = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/semdata/"

path_2_preanalysis_data = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part2_sem/preanalysis/"
path_2_preanalysis_fig = drive_letter + "/wyy/code_project/running_outcome/final_fig/SPDB/part2_sem/preanalysis/"

path_temp = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/temp/"

path_ng_fig = drive_letter + "/wyy/code_project/running_outcome/final_fig/ng_fig/"

mark_num = "25"
meta_name = "meta_data.csv"
list_color = ["#ee877c", "#8bd0e3", "#6abeae", "#808eaf", "#f7bba8", "#acb4cc", "#b5e0d5", "#e86462", "#a89687"]

In [4]:
from sklearn.ensemble import RandomForestRegressor

scoring = 'r2'
str_describe = 'lrsws7'
df_lr_sem = pd.read_csv(path_semdata + "sem_lrsws7_au_to_25_avg_full.csv")
path_lr_rfecv_data = drive_letter + '/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/lr_forecast/'

df_lr_sem['year'] = (df_lr_sem['year'] - 2000) / (2020 - 2000)

selected_features_rf = pd.read_csv(path_lr_rfecv_data + str_describe + "_rfecv_features_RFcv.csv")
selected_features_rf = selected_features_rf[selected_features_rf["Rank"] == 1]["Feature"].values

best_params = pd.read_csv(path_lr_rfecv_data + 'ml_cv_best.csv')

rf_params = best_params[best_params["model"] == "RF"].iloc[0]

if rf_params["param_max_depth"] == 'None':
    param_max_depth_rf = None
else:
    param_max_depth_rf = int(rf_params["param_max_depth"])
model_fish = RandomForestRegressor(
    max_depth=param_max_depth_rf,
    min_samples_leaf=int(rf_params["param_min_samples_leaf"]),
    min_samples_split=int(rf_params["param_min_samples_split"]),
    n_estimators=int(rf_params["param_n_estimators"])
)
X = df_lr_sem[selected_features_rf]
y = df_lr_sem['value']



In [4]:

import shap
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

model_fish.fit(X, y)

try:
    explainer_fish = shap.TreeExplainer(model_fish)
    shap_values_fish = explainer_fish(X)
    
    # 保存 SHAP 值到本地文件
    shap_save_path_fish = path_2_preanalysis_fig + 'shap_values_lrsws7.pkl'
    with open(shap_save_path_fish, 'wb') as f:
        pickle.dump(shap_values_fish, f)
    
    print("SHAP values calculated and saved successfully.")
except Exception as e:
    print(f"Error calculating or saving SHAP values: {e}")



SHAP values calculated and saved successfully.


In [ ]:
from matplotlib.colors import LinearSegmentedColormap
import shap
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
shap_save_path_fish = path_2_preanalysis_fig + 'shap_values_lrsws7.pkl'
try:
    with open(shap_save_path_fish, 'rb') as f:
        loaded_shap_values_fish = pickle.load(f)
    print("SHAP values loaded successfully.")
except Exception as e:
    print(f"Error loading SHAP values: {e}")
    loaded_shap_values_fish = None

In [6]:
# top 5
import numpy as np
import matplotlib.colors as mcolors

feature_importance = np.abs(loaded_shap_values_fish.values).mean(0)
feature_importance_order = np.argsort(feature_importance)[::-1]

top_5_features = feature_importance_order[:5]

top_5_shap_values = loaded_shap_values_fish.values[:, top_5_features]
top_5_feature_names = X.columns[top_5_features]
top_5_X = X.iloc[:, top_5_features]

shap.summary_plot(top_5_shap_values, top_5_X, plot_type="bar", show=False, color='#7d84a8', feature_names=top_5_feature_names)
plt.title("SHAP Summary Bar Plot (Top 5 Features)")

ax = plt.gca()
for p in ax.patches:
    width = p.get_width()
    ax.text(width, p.get_y() + p.get_height()/2, f'{width:.3f}', 
            ha='left', va='center')

plt.tight_layout()
plt.savefig(path_ng_fig + 'fig4_lrsws7_bar_top5.svg', bbox_inches='tight')
plt.close()

colors = ['#FFFF00', '#FF0000']
n_bins = 100
cmap = LinearSegmentedColormap.from_list('custom_cmap', colors, N=n_bins)

shap.summary_plot(top_5_shap_values, top_5_X, plot_type="dot", show=False, cmap=cmap, feature_names=top_5_feature_names, alpha=0.1, max_display=5)
plt.title("SHAP Summary Dot Plot (Top 5 Features)")
plt.tight_layout()
plt.savefig(path_ng_fig + 'fig_lrsws7_dot_top5.svg', bbox_inches='tight')
plt.close()

The figure layout has changed to tight
The figure layout has changed to tight


### sw

In [6]:
from sklearn.ensemble import RandomForestRegressor

scoring = 'r2'
str_describe2 = 'sws7'
df_sw_sem = pd.read_csv(path_semdata + 'sem_sw_s7_au_to_' + mark_num +'_avg.csv')
path_sw_rfecv_data = drive_letter + '/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/sw_forecast/'


df_sw_sem['year'] = (df_sw_sem['year'] - 2000) / (2020 - 2000)
selected_features_rf = pd.read_csv(path_sw_rfecv_data + str_describe2 + "_rfecv_features_RFcv.csv")
selected_features_rf = selected_features_rf[selected_features_rf["Rank"] == 1]["Feature"].values

best_params = pd.read_csv(path_sw_rfecv_data + 'ml_cv_best.csv')

rf_params = best_params[best_params["model"] == "RF"].iloc[0]

if rf_params["param_max_depth"] == 'None':
    param_max_depth_rf = None
else:
    param_max_depth_rf = int(rf_params["param_max_depth"])
model_sw = RandomForestRegressor(
    max_depth=param_max_depth_rf,
    min_samples_leaf=int(rf_params["param_min_samples_leaf"]),
    min_samples_split=int(rf_params["param_min_samples_split"]),
    n_estimators=int(rf_params["param_n_estimators"])
)

X2 = df_sw_sem[selected_features_rf]
y2 = df_sw_sem['value']



In [8]:

import shap
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


model_sw.fit(X2, y2)

try:
    explainer_sw = shap.TreeExplainer(model_sw)
    shap_values_sw = explainer_sw(X2)
    
    # 保存 SHAP 值到本地文件
    shap_save_path_sw = path_2_preanalysis_fig + 'shap_values_sws7.pkl'
    with open(shap_save_path_sw, 'wb') as f:
        pickle.dump(shap_values_sw, f)
    
    print("SHAP values calculated and saved successfully.")
except Exception as e:
    print(f"Error calculating or saving SHAP values: {e}")



SHAP values calculated and saved successfully.


In [ ]:

import shap
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
shap_save_path_sw = path_2_preanalysis_fig + 'shap_values_sws7.pkl'
try:
    with open(shap_save_path_sw, 'rb') as f:
        loaded_shap_values_sw = pickle.load(f)
    print("SHAP values loaded successfully.")
except Exception as e:
    print(f"Error loading SHAP values: {e}")
    loaded_shap_values_sw = None


In [10]:
# top 5
import numpy as np
import matplotlib.colors as mcolors

feature_importance = np.abs(loaded_shap_values_sw.values).mean(0)
feature_importance_order = np.argsort(feature_importance)[::-1]

top_5_features = feature_importance_order[:5]

top_5_shap_values = loaded_shap_values_sw.values[:, top_5_features]
top_5_feature_names = X2.columns[top_5_features]
top_5_X2 = X2.iloc[:, top_5_features]

# Summary plot (bar)
shap.summary_plot(top_5_shap_values, top_5_X2, plot_type="bar", show=False, color='#84aeb8', feature_names=top_5_feature_names)
plt.title("SHAP Summary Bar Plot (Top 5 Features)")

# Add value labels
ax = plt.gca()
for p in ax.patches:
    width = p.get_width()
    ax.text(width, p.get_y() + p.get_height()/2, f'{width:.3f}', 
            ha='left', va='center')

plt.tight_layout()
plt.savefig(path_ng_fig + 'fig4_sws7_bar_top5.svg', bbox_inches='tight')
plt.close()


# Create a custom colormap from white to red
colors = ['#FFFF00', '#FF0000']
n_bins = 100
cmap = LinearSegmentedColormap.from_list('custom_cmap', colors, N=n_bins)

shap.summary_plot(top_5_shap_values, top_5_X2, plot_type="dot", show=False, cmap=cmap, feature_names=top_5_feature_names, alpha=0.1)
plt.title("SHAP Summary Dot Plot (Top 5 Features)")
plt.tight_layout()
plt.savefig(path_ng_fig + 'fig_sws7_dot_top5.svg', bbox_inches='tight')
plt.close()

The figure layout has changed to tight
The figure layout has changed to tight
